# 데이터 전처리 과정
### 사용 라이브러리

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time
import re

# 지리적 계산 라이브러리
from geopy.distance import geodesic
from haversine import haversine

# HTTP 요청 및 API 라이브러리
import requests
import googlemaps

# 멀티스레딩 및 병렬 처리 라이브러리
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed

#### 사용 API KEY
본인 API KEY를 입력하여 사용하시오

In [2]:
GOOGLE_API_KEY = ''  # geocoding, direction api 사용
KAKAO_API_KEY = ''  # 지오코딩의 경우 카카오 api를 우선적으로 사용하고 카카오 맵 api가 못하는 주소에 대해서 구글 geocoding api 사용

# 1. 은행 데이터

In [2]:
bank = pd.read_csv('codefilex.csv')  # 금융결제원 - 금융회사코드 조회
bank['address'] = bank['주소'].apply(lambda x: x.replace("\u3000", " "))
bank = bank[bank['address'].str.contains('전라남도')].iloc[:100] # iloc 지우기
bank.reset_index(inplace=True, drop=True)

In [3]:
bank

,은행코드,은행명,점포명,전화번호,팩스,우편번호,주소,구분,폐쇄관리지점,address
0,14203,한국,목포 본부,061 241 1000,061 243 8066,58723,전라남도 목포시 영산로 １０９ ...,정상,NaN,전라남도 목포시 영산로 １０９ ...
1,28202,산업,여수（지）,061 690 8500,061 682 0165,59677,전라남도 여수시 시청로 ３０ ...,정상,NaN,전라남도 여수시 시청로 ３０ ...
2,28901,산업,목포（지）,061 280 5400,061 285 0353,58690,전라남도 목포시 백년대로 ３０６ ...,정상,NaN,전라남도 목포시 백년대로 ３０６ ...
3,31914,기업,목포（지）,061 282 0202,0505075 0191,58688,전라남도 목포시 옥암로 ２５ ...,정상,NaN,전라남도 목포시 옥암로 ２５ ...
4,31927,기업,순천（지）,061 07265061,0505075 0192,57967,전라남도 순천시 충효로 １５２ ...,정상,NaN,전라남도 순천시 충효로 １５２ ...
...,...,...,...,...,...,...,...,...,...,...
95,91310,수협은행,목포수협채권관리실,061 240 0290,061 240 0219,58623,전라남도 목포시 청호로２１９번길 ５０ （죽교동） 수협 ...,정상,NaN,전라남도 목포시 청호로２１９번길 ５０ （죽교동） 수협 ...
96,100298,ＮＨ농협은행,ａＴ본사,061 337 7101,061 337 7105,58217,전라남도 나주시 문화로 ２２７ 한국농수산식품유통공사１층 ...,정상,NaN,전라남도 나주시 문화로 ２２７ 한국농수산식품유통공사１층 ...
97,100696,ＮＨ농협은행,전남도청（출）,061 287 4700,061 287 4701,58564,전라남도 무안군 삼향읍 오룡길 １ ...,정상,NaN,전라남도 무안군 삼향읍 오룡길 １ ...
98,101200,ＮＨ농협은행,여수센트럴,061 654 3101,061 654 3103,59708,전라남도 여수시 여서１로 ５３ １층 ...,정상,NaN,전라남도 여수시 여서１로 ５３ １층 ...


In [4]:
def kakao_geocoding_single(query, api_key):
    """단일 주소에 대해 지오코딩 작업을 수행하는 함수
    """
    try:
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        headers = {"Authorization": f"KakaoAK {api_key}"}
        params = {"query": query}

        response = requests.get(url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        result = response.json()

        if result['documents']:
            longitude = result['documents'][0]['x']
            latitude = result['documents'][0]['y']
        else: # 검색 결과 없을 때 위경도 값 None으로 설정
            longitude = None
            latitude = None
    except Exception as e:
        print(f"Error: {e}")
        longitude = None
        latitude = None

    return longitude, latitude

def kakao_geocoding(house_df, api_key):
    """카카오 맵 api로 지오코딩 (멀티스레딩 버전)
    """
    queries = house_df['address'].tolist()  # 주택 주소 목록
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # executor.map에서 반환되는 결과를 캡처하여 tqdm으로 진행 상황을 표시
        results = list(tqdm(executor.map(lambda x: kakao_geocoding_single(x, api_key), queries), total=len(queries), desc="Geocoding"))

    # 결과를 사용하여 데이터프레임 업데이트
    for index, (longitude, latitude) in zip(house_df.index, results):
        house_df.at[index, 'longitude'] = longitude
        house_df.at[index, 'latitude'] = latitude

    return house_df

# 구글 맵 api
def address_to_coordinates(api_key, address):
    # Google Maps 클라이언트 생성
    gmaps = googlemaps.Client(key=api_key)
    
    # 주소를 좌표로 변환
    geocode_result = gmaps.geocode(address)
    
    # 변환된 결과에서 좌표 추출
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        return latitude, longitude
    else:
        return None, None

In [5]:
bank_geo_kakao = kakao_geocoding(bank, KAKAO_API_KEY) # 카카오 api로 지오코딩

Geocoding: 100%|██████████| 100/100 [00:13<00:00,  7.68it/s]


In [6]:
# 구글 맵 api 지오코딩
bank_geo_google = bank_geo_kakao[bank_geo_kakao['latitude'].isna()] # 카카오맵 api로 지오코딩 안된 주소에 한해서 구글 맵 api로 지오코딩 수행
bank_geo_kakao = bank_geo_kakao[~bank_geo_kakao['latitude'].isna()] # 카카오맵 api 지오코딩 안된 주소 제거
bank_geo_google.reset_index(inplace=True, drop=True)

# 새로운 열을 생성하여 좌표를 저장할 예정
bank_geo_google['latitude'] = None
bank_geo_google['longitude'] = None

# 각 주소에 대해 구글 맵스 API를 적용하여 좌표를 찾고 DataFrame에 추가
for index, row in tqdm(bank_geo_google.iterrows(), total=len(bank_geo_google)):
    address = row['address']
    latitude, longitude = address_to_coordinates(GOOGLE_API_KEY, address)
    bank_geo_google.at[index, 'latitude'] = latitude
    bank_geo_google.at[index, 'longitude'] = longitude

C:\Users\user\AppData\Local\Temp\ipykernel_6012\4085765651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_geo_google['latitude'] = None
C:\Users\user\AppData\Local\Temp\ipykernel_6012\4085765651.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_geo_google['longitude'] = None
100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


In [7]:
# 카카오 api 지오코딩 + 구글 api 지오코딩
bank_df = pd.concat([bank_geo_kakao, bank_geo_google])
bank_df.dropna(inplace=True, subset='latitude') #주소 지오코딩이 안되는 경우 제거
bank_df = bank_df[bank_df['폐쇄관리지점'].isna()] # 폐쇄 지점 제거
bank_df.reset_index(inplace=True, drop=True)

# latitude와 longitude 열을 float로 변환
bank_df['latitude'] = bank_df['latitude'].astype(float)
bank_df['longitude'] = bank_df['longitude'].astype(float)

# 한국 영토의 위경도 범위 설정
min_latitude = 33
max_latitude = 38
min_longitude = 124
max_longitude = 132

# 위경도 범위를 벗어나는 행 제거(한국 외의 외국 은행 제거)
bank_df_ko = bank_df[(bank_df['latitude'] >= min_latitude) & (bank_df['latitude'] <= max_latitude) &
        (bank_df['longitude'] >= min_longitude) & (bank_df['longitude'] <= max_longitude)]

bank_df_ko.to_csv('bank_전남.csv',encoding='cp949')

# 2. 주택

In [8]:
# 개별 주택 가격 정보
single_jeonnam = pd.read_csv('개별주택가격정보_전라남도_20240125.csv',encoding='cp949')
single_df = single_jeonnam[['법정동코드', '법정동명', '특수지구분명', '지번']].iloc[:50].copy() # head 지우기
single_df_m = single_jeonnam[['법정동코드', '법정동명', '특수지구분명', '지번']].iloc[394028:394032] # 산 제대로 들어가는지만 확인
single_final = pd.concat([single_df, single_df_m])

# 공동 주택 가격 정보
public_jeonnam = pd.read_csv('공동주택가격정보_전라남도_20240125.csv',encoding='cp949')
public_df = public_jeonnam[['법정동코드', '법정동명', '특수지구분명', '지번', '공동주택구분명']].iloc[:50].copy() #head 지우기

jeonnam_house = pd.concat([public_df, single_final])

C:\Users\user\AppData\Local\Temp\ipykernel_6012\1197982495.py:2: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  single_jeonnam = pd.read_csv('개별주택가격정보_전라남도_20240125.csv',encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_6012\1197982495.py:8: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  public_jeonnam = pd.read_csv('공동주택가격정보_전라남도_20240125.csv',encoding='cp949')


In [9]:
def clean_address(row):
    """ 각 집에 대해서 전부 지오코딩을 수행하면 너무 많아서 같은 지번으로 묶는 작업
    ex) 전라남도 나주시 다시면 월태리 195-2 -> 전라남도 나주시 다시면 월태리 195
    """
    if pd.isnull(row['지번']):
        return row['법정동명']
    elif '-' in row['지번']:
        # '-'가 있는 경우 '-' 앞의 값을 사용하여 'address_clean' 열 생성
        return row['법정동명'] + ' ' + row['지번'].split('-')[0]
    else:
        return row['법정동명'] + ' ' + row['지번']

In [10]:
# 주소 전처리
jeonnam_house['address_clean'] = jeonnam_house.apply(clean_address, axis=1)
jeonnam_house['address'] = jeonnam_house['법정동명']+' '+jeonnam_house['지번']

jeonnam_house = jeonnam_house.drop_duplicates(subset='address_clean') #지번으로 중복제거
jeonnam_house.reset_index(inplace=True, drop=True)
jeonnam_house.fillna({'공동주택구분명':'개인주택'}, inplace=True)

house_geo_kakao = kakao_geocoding(jeonnam_house, KAKAO_API_KEY) # 카카오 api로 지오코딩

Geocoding: 100%|██████████| 12/12 [00:01<00:00,  7.82it/s]


In [11]:
# 산 지역이나 카카오맵 api가 지오코딩 하지 못한 주소에 대해서 구글 맵 api 적용
house_geo_google = house_geo_kakao[house_geo_kakao['latitude'].isna()].copy() 
house_geo_google.reset_index(inplace=True, drop=True)

# 새로운 열을 생성하여 좌표를 저장할 예정
house_geo_google['latitude'] = None
house_geo_google['longitude'] = None

# 각 주소에 대해 구글 맵스 API를 적용하여 좌표를 찾고 DataFrame에 추가
for index, row in tqdm(house_geo_google.iterrows(), total=len(house_geo_google)):
    address = row['address_clean']
    latitude, longitude = address_to_coordinates(GOOGLE_API_KEY, address)
    house_geo_google.at[index, 'latitude'] = latitude
    house_geo_google.at[index, 'longitude'] = longitude
    
# 카카오 api 지오코딩 + 구글 api 지오코딩 
house_geo = house_geo_kakao[house_geo_kakao['latitude'].notna()] # 카카오 api로 지오코딩 된 주소
house_geo_df = pd.concat([house_geo, house_geo_google], axis = 0) 
house_geo_df.reset_index(inplace=True, drop=True)

house_geo_df.to_csv('./data/jeonnam_geo.csv', encoding='cp949', index=False)

0it [00:00, ?it/s]


# 3. 거리 및 이동시간 측정



In [12]:
bank_jeonnam = pd.read_csv('./data/bank_전남.csv',encoding='cp949')
public_jeonnam = pd.read_csv('./data/jeonnam_geo.csv',encoding='cp949')

In [13]:
# 각 주택과 가장 가까운 은행 코드 구하기
public_jeonnam['closeness'] = None
public_jeonnam['distance'] = None

for index, row in tqdm(public_jeonnam.iterrows(), total=len(public_jeonnam)):
    target_latitude = row['latitude']
    target_longitude = row['longitude']
    
    # 각 좌표와의 거리 계산
    distances = []
    for _, bank_row in bank_jeonnam.iterrows():
        coordinates = (bank_row['latitude'], bank_row['longitude'])
        distance = geodesic((target_latitude, target_longitude), coordinates).meters
        distances.append(distance)

    # 가장 가까운 좌표의 은행코드 값과 거리 가져오기
    min_index = distances.index(min(distances))
    closest_bank_code = bank_jeonnam.loc[min_index, '은행코드']
    closest_distance = distances[min_index]
    
    public_jeonnam.at[index, 'closeness'] = closest_bank_code
    public_jeonnam.at[index, 'distance'] = closest_distance

100%|██████████| 12/12 [00:00<00:00, 55.53it/s]


In [14]:
def get_duration(gmaps, origin, destination, departure_time, index):
    try:
        directions = gmaps.directions(origin, destination, mode="transit", departure_time=departure_time)
        if directions:
            duration = directions[0]['legs'][0]['duration']['text']
            return index, duration
        else:
            return index, None
    except Exception as e:
        print(f"Error for index {index}: {e}")
        return index, None

def distance_measure(df, bank_df, api_key):
    """
    주택 기준 가장 가까운 은행까지 걸리는 시간 측정
    구글 맵 direction api 활용
    교통수단으로 걸리는 시간 결과 반환
    
    너무 가까워서 교통수단이 필요없거나 길이 없는 경우 결과값 none
    
    출발시간: 2024-05-27(월) 오전 10시
        구글 맵의 경우 현재 날짜 기준 1주 이전 날짜로 출발시간을 설정하면 반환하지 못함
    """

    departure_time = datetime(2024, 5, 27, 10, 0, 0)

    # Google Maps 클라이언트 생성
    gmaps = googlemaps.Client(key=api_key)

    # 결과를 저장할 리스트 초기화
    durations = [None] * len(df)

    # 요청 속도 제어를 위한 변수 초기화
    max_requests_per_minute = 3000
    max_requests_per_second = max_requests_per_minute / 60
    request_interval = 1 / max_requests_per_second
    request_count = 0
    start_time = time.time()

    # 멀티스레딩 사용
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {
            executor.submit(
                get_duration, 
                gmaps, 
                row['address'] if pd.notna(row['address']) else row['address_clean'], 
                (bank_df.loc[bank_df['은행코드'] == row['closeness'], 'latitude'].values[0],
                 bank_df.loc[bank_df['은행코드'] == row['closeness'], 'longitude'].values[0]),
                departure_time, 
                index
            ): index for index, row in df.iterrows()
        }

        for future in tqdm(as_completed(future_to_index), total=len(df)):
            index, duration = future.result()
            durations[index] = duration

            # 요청 속도 제어
            request_count += 1
            if request_count >= max_requests_per_second:
                elapsed_time = time.time() - start_time
                if elapsed_time < 1:
                    time.sleep(1 - elapsed_time)
                start_time = time.time()
                request_count = 0

    # DataFrame에 결과 추가
    df['transit_duration'] = durations
    return df

In [15]:
jeonnam_transition = distance_measure(public_jeonnam, bank_jeonnam, GOOGLE_API_KEY)

100%|██████████| 12/12 [00:02<00:00,  5.74it/s]


In [16]:
def hour_to_mins(duration):
    if not isinstance(duration, str):
        return 0
    
    # ISO 8601 Duration pattern
    pattern = re.compile(r'(?:(\d+)\s*hours?)?\s*(?:(\d+)\s*mins?)?')
    match = pattern.match(duration)
    
    if not match:
        return 0
    
    hour = int(match.group(1) or 0)
    mins = int(match.group(2) or 0)
    
    total_mins = hour * 60 + mins
    
    return total_mins

# apply 함수로 열에 적용
jeonnam_transition['total_mins'] = jeonnam_transition['transit_duration'].apply(hour_to_mins)

In [ ]:
jeonnam_transition_notna = jeonnam_transition[jeonnam_transition['transit_duration'].notna()] # direction api로 추출되지 않는 주소가 있어서 제거

In [17]:
jeonnam_transition_notna.to_csv('./data/jeonnam_transition.csv',encoding='cp949',index=False)

# 4. 반경 내 은행 개수 구하기

In [18]:
jeonnam_transition = pd.read_csv('./data/jeonnam_transition.csv',encoding='cp949')
bank_jeonnam = pd.read_csv('./data/bank_전남.csv',encoding='cp949')

In [19]:
# 반경 범위 설정
radius_ranges = [1, 1.5, 2, 2.5, 3, 5, 10]

# 거리 범위에 따른 컬럼명 설정
column_names = [f"{radius}km_radius" for radius in radius_ranges]

# 각 컬럼을 0으로 초기화하여 sp에 추가
for column_name in column_names:
    jeonnam_transition[column_name] = 0

# 각 출발지마다 반경 내 도착지 개수 계산하여 컬럼에 저장
for index, row in tqdm(jeonnam_transition.iterrows(), total=len(jeonnam_transition)):
    origin = (row['latitude'], row['longitude'])
    
    for index_bank, row_bank in bank_jeonnam.iterrows():
        destination = (row_bank['latitude'], row_bank['longitude'])
        distance_km = haversine(origin, destination, unit='km')
        
        # 반경 범위 내에 있는지 확인하여 해당 컬럼의 값을 증가
        for radius, column_name in zip(radius_ranges, column_names):
            if distance_km <= radius:
                jeonnam_transition.at[index, column_name] += 1

100%|██████████| 12/12 [00:00<00:00, 142.82it/s]


In [21]:
jeonnam_transition.to_csv('./data/jeonnam_transition.csv',encoding='cp949', index=False)

# 5. 인구 데이터랑 합치기


In [24]:
# 이전 데이터 불러오기
jeonnam_transition = pd.read_csv('./data/jeonnam_transition.csv',encoding='cp949')

In [25]:
# 필요한 컬럼만 추출
jeonnam_transition = jeonnam_transition[['address', 'latitude', 'longitude', 'closeness', 'distance', 'total_mins', 
                                        '1km_radius', '1.5km_radius', '2km_radius', '2.5km_radius', '3km_radius', '5km_radius', '10km_radius']]

In [26]:
# 전처리 완료한 전라남도 인구 데이터 불러오기
df_pop = pd.read_csv('./data/jeonnam_pop.csv', encoding='cp949')

In [27]:
# 인구데이터를 병합
df_final = pd.merge(jeonnam_transition, df_pop, left_on='address', right_on='주소', how='inner')

In [29]:
# 최종 데이터를 Parquet 파일로 저장
df_final.to_parquet('./data/jeonnam_final.parquet', index=False)